# The Easy "Hard" Way: Cythonizing

[Cython](http://cython.org/) is a **compiler** and a **programming language** used to generate C extensions for Python.

The Cython language is a Python/C [creole](https://en.wikipedia.org/wiki/Creole_language) which is essentially Python with some additional keywords for specifying static data types. It looks something like this:

```cython
def cython_sum(int n):
    cdef float s = 0.0
    cdef int i
    for i in range(n):
        s += i
    return s
```

You can write in this language, then use the Cython compiler to *transpile* to efficient C code which can in turn be compiled into a Python extension module. This extension module can be imported like a normal Python module, but it is backed by compiled C, offering potentially drastic speedups over equivalent functions written in pure Python.

The other major use case for Cython -- the one we will focus on here -- is writing wrappers around existing C code so that the functions therein can be made available in an extension module as described above. We will use this technique to make the SymPy-generated C code accessible to Python for use in SciPy's `odeint`.

**Learning Objectives**

After this lesson, you will be able to:

- write a simple Cython function and run it a Jupyter notebook using the `%%cython` magic command
- use Cython to wrap a function implemented in plain C and make it callable from Python
- use the SymPy `codegen` function to output compilable C code
- wrap `codegen`-generated code with Cython, compile it into an extension module, and call it from Python
- use SymPy's `autowrap` function to do all of this behind the scenes
- pass a custom code printer to `autowrap` to make use of an efficient C library

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sympy as sm
sm.init_printing()

## 1. Introduction to Cython

Cython is used in two scenarios:

1. writing Cython computation code (i.e. Python with statically typed variables) to generate fast extension modules
2. writing Cython wrappers around existing C code

In both cases, efficient C code can be generated and made available via import from Python.

### Writing Cython Code

In the first use case, you write in the Cython language, compile this into CPython C code, then compile it into a Python extension module. There are several ways to ways to go about the compilation process, and in many cases, Cython's tooling makes it fairly simple. For example, Jupyter notebooks can make use of a `%%cython` magic command that will do all of compilation in the background for us. To make use of it, we need to load the `cython` extension.

In [ ]:
%load_ext cython

Now we can write a Cython function. Note that the `--annotate` (or `-a`) flag of the `%%cython` magic command will produce an interactive annotated printout of the Cython code, allowing us to see the C code that is generated.

In [ ]:
%%cython --annotate
def cython_fib(int n):
    cdef int i
    cdef double a = 0.0, b = 1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [ ]:
%timeit cython_fib(100)

**Exercise**: remove the type declarations to turn the Cython code above into a plain Python function and use `%timeit` to compare performance

In [ ]:
def python_fib(n):
    a, b = 0.0, 1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [ ]:
%timeit python_fib(100)

To see a bit more about writing Cython and its potential performance benefits, see [this Cython examples notebook](cython-examples.ipynb).

## 2. Wrapping C Code using Cython

Our main goal in using Cython is to make numerical code generated by SymPy available to Python. To achieve this, we'll focus on the second use case for Cython -- wrapping existing C code. Wrapping a set of C functions involves wrapping existing C code with a fairly simple Cython script that specifies the Python interface to the C functions. This script must do two things:

1. specify the function signatures as found in the C header file
2. implement the Python interface to the functions by wrapping them

### Example

Imagine you have the following C code:

```c
// file: fib.c
double fib(int n) {
    int i;
    double a = 0.0, b = 1.0, tmp;
    for (i = 0; i < n; i++) {
        tmp = a; a = a + b; b = tmp;
    }
    return a;
}
```

```c
// file: fib.h
double fib(int n);
```

To wrap this library, we just need to create a Cython file like so:

```cython
# file: fib.pyx
cdef extern from "fib.h":
    double fib(int n)
    
def fib(n):
    return fib(n)
```

Once this is compiled into an extension module, we can import it just like any other Python module. When we use the module's `fib` function, the *compiled C code* is performing the computation, potentially offering major speedups.

### Using `codegen()` to Generate C Code

In the previous notebook focusing on generating C code using SymPy's codegen capabilities, we saw how to form a Jacobian from a system of ODEs, then output C code that can numerically evaluate the right hand side of the ODEs themselves as well as the Jacobian, given the current state vector. Here we'll do that again so that we can wrap it in Cython for use in `scipy.integrate.odeint`.

Start by importing the system of ODEs and compute the Jacobian analytically.

In [ ]:
from scipy2017codegen.chem import load_large_ode

rhs_of_odes, states = load_large_ode()
jac_of_odes = rhs_of_odes.jacobian(states)

Now we'll use `sympy.utilities.codegen.codegen` to output C source and header files which can compute the right hand side (RHS) of the ODEs and the Jacobian.

In [ ]:
from sympy.utilities.codegen import codegen

# form equations so we can put the OutputArgument in the argument_sequence
dY = sm.MatrixSymbol('dY', len(states), 1)
dY_eq = sm.Eq(dY, rhs_of_odes)
t = sm.symbols('t')

codegen([('c_odes', dY_eq)], 'c', argument_sequence=[*list(states), t, dY], to_files=True)

Let's take a look at the generated C code.

In [ ]:
%cat c_odes.c

### Wrapping the Generated Code with Cython

Now we want to wrap the function that was generated `c_odes` with a Cython function. Let's output just the header `c_odes.h` to see just the function signature.

In [ ]:
%cat c_odes.h

So this function signature is what we need to wrap using Cython. A function is provided in our tutorial package to simplify the build configuration so that our `c_odes.{c,h}` files are found, built, and linked correctly with our wrapper. It takes the name of the module we're creating, then some keword arguments:

- `sources`: C source files that need to be built
- `include_dirs`: locations of header files -- the numpy headers are needed for use of numpy in our wrapper code

In [ ]:
from scipy2017codegen.templates import render_pyxbld
render_pyxbld('cy_odes', sources=['c_odes.c'], include_dirs=[np.get_include()])

**Exercise**: Fill in the wrapper implementation below.

In [ ]:
%%cython_pyximport cy_odes
import numpy as np
cimport numpy as np

cdef extern from "c_odes.h":
    void c_odes(double y0, double y1, double y2, double y3, double y4,
                double y5, double y6, double y7, double y8, double y9,
                double y10, double y11, double y12, double y13, double t,
                double *dY)
    
def cy_odes(y, t):
    cdef np.ndarray[np.float64_t, ndim=1, mode='c'] dY
    dY = np.empty(y.size, dtype=np.float64)
    c_odes(y[0], y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8], y[9],
           y[10], y[11], y[12], y[13], t, <double *> dY.data)
    return dY

Now use `odeint` to integrate the equations and plot the results to check that it worked.

In [ ]:
from scipy.integrate import odeint
from scipy2017codegen.chem import watrad_init, watrad_plot

y_init, t_vals = watrad_init()
y_vals = odeint(cy_odes, y_init, t_vals)
watrad_plot(t_vals, y_vals)

## 3. Generating and Compiling a C Extension Module Automatically

Here we'll use SymPy's `autowrap` function to automatically generate the C code for an expression, generate a Cython wrapper, compile the wrapper into an extension module, and make the function available as a callable.

In [ ]:
from sympy.utilities.autowrap import autowrap

# need to specify input args so they are in correct order
args = list(states)
args.append(t)

f_rhs = autowrap(rhs_of_odes, args=args, backend='cython', tempdir='./autowraptmp')

`autowrap` essentially does exactly what we just did manually: generate C code to evaluate the right hand side of the system of ODEs, write a Cython wrapper, compile the C code and wrapper into an extension module, and import it for use in Python.

**Exercise**: just like the `codegen`, `autowrap` generates a C function that takes each of the state variables as separate arguments, as seen in the generated Cython wrapper file seen above. In order to use the callable returned by `autowrap` within `odeint`, write a Python wrapper that takes the state vector `y` and the current time `t` and uses `f_rhs` to compute the derivatives.

In [ ]:
def rhs_eval(y, t):
    yout = f_rhs(*y, t)
    return yout[:, 0]

y_vals = odeint(rhs_eval, y_init, t_vals)
watrad_plot(t_vals, y_vals)

### Using a Custom Library

In our set of ODEs, there are quite a few instances of $y_i^2$, where $y_i$ is one of the 14 state variables.

In [ ]:
from sympy.utilities.codegen import C99CodeGen
from sympy.printing.ccode import C99CodePrinter

class CustomPrinter(C99CodePrinter):
    def _print_Pow(self, expr):
        return 'fastpow({}, {})'.format(expr.base, expr.exp)

printer = CustomPrinter()
gen = C99CodeGen(printer=printer)
gen.preprocessor_statements.append('#include "fastpow.h"')

import scipy2017codegen
import os
fastapprox_dir = os.path.join(os.path.dirname(scipy2017codegen.__file__), 'fastapprox')

f_rhs = autowrap(rhs_of_odes, args=args, code_gen=gen, backend='cython',
                 tempdir='./autowraptmp2', include_dirs=[fastapprox_dir])

In [ ]:
y_vals, info = odeint(rhs_eval, y_init, t_vals, full_output=True)
watrad_plot(t_vals, y_vals)

In [ ]:
info